<a href="https://colab.research.google.com/github/Ian306/Regresion-logistica-con-SGD/blob/main/Regresion_logistica_con_SGD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción al Aprendizaje Automático &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; UANL - FCFM
---
## Regresion logistica con Descenso de Gradiente Estocástico

**Alumnos**
- Ian Mauricio Saucedo Alemán &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 1868954
- Jose Francisco Gongora Rangel &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 1855819

_Librerías_

In [ ]:
# Introduccion al Aprendizaje Automatico

# Importamos las librerias que utilizaremos
import numpy as np
import math 
from math import e
import pandas as pd
import plotly.express as px
import random
from decimal import *
import warnings
warnings.filterwarnings("ignore")

1. Creamos 100 valores aleatorios uniformemente distribuidos para x entre -10 y 10 
2. Asignar y a cada valor de x usando el criterio: 
  - Generar un número aleatorio uniforme R entre 0 y 1
    - Si x >= 2 y R<=0.9 -> y=1 
    - Si x >= 2 y R>0.9 -> y=0 
    - Si x < 2 y R<=0.9 -> y=0 
    - Si x < 2 y R>0.9 -> y=1

In [ ]:
N = 100
x = np.random.uniform(low=-10,high=10,size=(N,1))
R = np.random.uniform(low=0,high=1,size=(N,1))
y = []
for i in range(len(x)):
  if ((x[i] >= 2 and R[i] <= 0.9) or (x[i] < 2 and R[i] > 0.9)):
    y.append(1)
  elif ((x[i] >= 2 and R[i] > 0.9) or (x[i] < 2 and R[i] <= 0.9)):
    y.append(0)
df = pd.DataFrame.from_dict({
    'x': np.squeeze(x),
    'y': np.squeeze(y)
})
print(df)

           x  y
0  -1.029715  0
1   4.886902  1
2   9.485060  1
3  -3.095156  0
4  -8.136097  0
..       ... ..
95 -4.022581  0
96 -7.193008  1
97  8.160336  1
98  1.406292  0
99 -8.937655  0

[100 rows x 2 columns]


Grafica de dispersión de nuestros puntos **x** y **y**

In [ ]:
px.scatter(df, y = 'y', x = 'x')

Para graficar correctamente como se comportan las x debemos de ordenarlas

In [ ]:
px.line(df, x='x', y='y')

In [ ]:
df = df.sort_values(by='x')
print(df)

           x  y
7  -9.882849  1
29 -9.819899  0
53 -9.444159  0
31 -9.402056  0
89 -9.296016  1
..       ... ..
25  8.635798  1
79  8.918591  1
2   9.485060  1
80  9.520844  1
68  9.869111  1

[100 rows x 2 columns]


Comportamiento de las x obtenidas

In [ ]:
px.line(df, x='x', y='y')

Nuestros valores de m y b seran 2 y 0.9 respectivamente

In [ ]:
m = 2
b = 0.9
yhat = 1 / (1 + e**(-x*m + b))

Entonces podemos crear un dataframe con los valores de y predecidos con nuestra m y b

In [ ]:
df_predict = pd.DataFrame({
    'x':np.squeeze(x),
    'y':np.squeeze(y),
    'yhat':np.squeeze(yhat)
})
df_predict.sort_values(by=['x'],inplace=True)
df_predict = df_predict.round(10)
df_predict

,x,y,yhat
7,-9.882849,1,1.100000e-09
29,-9.819899,0,1.200000e-09
53,-9.444159,0,2.500000e-09
31,-9.402056,0,2.800000e-09
89,-9.296016,1,3.400000e-09
...,...,...,...
25,8.635798,1,9.999999e-01
79,8.918591,1,1.000000e+00
2,9.485060,1,1.000000e+00
80,9.520844,1,1.000000e+00


Grafica de la regresion logistica con los valores de m = 2 y b = 0.9

In [ ]:
df_predict_plot = df_predict.melt(id_vars=['x'])
px.line(df_predict_plot,x='x',y='value',color='variable')

### Binary Cross Entropy Loss function 
$ H_p(q) = - \frac{ 1 }{N} \sum_{i=1}^N y_i \cdot log(p(y_i)) + (1 - y_i)\cdot log(1 - p(y_i)) \\p(y_i) = \frac{ 1 }{ (1 + e^{-x*m+b}) }
$


In [ ]:
def loss(y, yhat):
  return -np.mean(y * np.log10(yhat) + (1 - y) * np.log10(1 - yhat))

In [ ]:
def J(m, b, x, y):
  yhat = 1 / (1 + e**(-x*m + b))
  return loss(y, yhat)

El valor de la funcion de perdida con los valores de m = 2 y b = 0.9

In [ ]:
J(2, 0.9, df_predict['x'], df_predict['y'])

0.6882796907239002

Creamos las funciones que nos regresaran el gradiente de m y b

In [ ]:
def g_m(x,y,m,b):
  # Calcular gradiente de J(m,b) de m evaluado en m,b 
  # usando los datos x,y
  #gm = (-1/np.log(10))*np.mean((x * e**(-x*m + b)/(1+ e**(-x*m + b)))*(2*yhat - 1))
  gm = -np.mean((e**(-x*m+b)*y*x - x*(-y+1))/(np.log(10)*(1 + e**(-x*m+b))))
  if (math.isnan(gm)): 
    return 1
  return gm

def g_b(x,y,m,b):
  # Calcular gradiente de J(m,b) de b evaluado en m,b 
  # usando los datos x,y
  #gb = (-1/np.log(10))*np.mean((e**(-x*m + b)/(1+ e**(-x*m + b)))*(-2*yhat + 1))
  gb = -np.mean(((-y*e**(b-x*m))/(np.log(10)*(1 + e**(-x*m + b)))) + ((1-y)*(1/np.log(10))*(1/(1-(1/(1+e**(-x*m + b)))))*((e**(b-x*m))/((e**(b-x*m) + 1)**2))))
  if (math.isnan(gb)): 
    return 1
  return gb
  

Reducimos dimensionalidad de arrays para manejarlos mejor

In [ ]:
x = np.squeeze(x)
y = np.squeeze(y)

In [ ]:
print(g_m(x,y,2,0.9))
print(g_b(x,y,2,0.9))

0.3353828020808804
-0.022485989032475794


Este es el metodo que utilizaremos para obtener n puntos aleatorios de nuestras x y y, y poder usarlos en el SGD

In [ ]:
import random
xy_random = [[],[]]
for i in range(20):
  rand = np.random.randint(0,len(x))
  xy_random[0].append(x[rand])
  xy_random[1].append(y[rand])
xy_random = np.squeeze(xy_random)
xy_random

array([[-1.02971458, -1.02971458, -9.88284887, -8.64041097,  1.29957397,
        -6.90582837, -4.10091933, -8.51417003,  5.88316427,  4.52979389,
         1.5050032 ,  3.53866065, -3.14441124, -7.43839014,  8.1603363 ,
         2.94383912,  4.88690195,  4.16540954,  0.88879846,  3.13269383],
       [ 0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  1.        ,  1.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  0.        ,  1.        ]])

### Descenso por Gradiente

In [ ]:
# Inicializar los parametros aleatoriamente
m = np.random.randint(0,5)
b = np.random.randint(0,5)
n_iter = 10
k = 0 
lr = 0.1
while k<=n_iter:
  # Calcular gradientes
  gr_b = g_b(x,y,m,b)
  gr_m = g_m(x,y,m,b)
  # Mover los parametros en la direccion opuesta del gradiente
  m = m- lr*gr_m
  b = b- lr*gr_b
  k = k + 1
#np.seterr(divide = 'ignore') 
print(m,b)
print(J(m, b, x, y))

3.637695531825279 4.009587819519664
1.1892978668081378


### Descenso por Gradiente Estocástico

In [ ]:
def SGD(n_iter, lr, batch):
  # Inicializar los parametros aleatoriamente
  m = np.random.randint(0,5)
  b = np.random.randint(0,5)
  k = 0 
  tol = 0.01
  while k<=n_iter:
    Jant = J(m,b,x,y)
    # Calcular gradientes (sobre un subconjunto de datos aleatorio) 
    xy_random = [[],[]]
    for i in range(batch):
      rand = np.random.randint(0,len(x))
      xy_random[0].append(x[rand])
      xy_random[1].append(y[rand])
    xy_random = np.squeeze(xy_random)
    gr_b = g_b(xy_random[0],xy_random[1],m,b)
    gr_m = g_m(xy_random[0],xy_random[1],m,b)
    # Mover los parametros en la direccion opuesta del gradiente
    m = m- lr*gr_m
    b = b- lr*gr_b
    Jnew = J(m,b,x,y)
    if np.abs(Jnew-Jant) <= tol:
      k = n_iter
    k = k + 1
  np.seterr(divide = 'ignore') 
  return m, b, J(m,b,x,y)
m,b, j = SGD(100, 0.01, 20)
m,b,j

(2.9995533223179374, 3.0003375996571564, 0.9882532918376632)

Aqui podemos graficar la prediccion con los m y b obtenidos en el paso anterior

In [ ]:
def plot_logreg(m, b):
  y_predicted = 1 / (1 + e**(-x*m + b))
  df_predicted = pd.DataFrame({
      'x':np.squeeze(x),
      'y':np.squeeze(y),
      'y_predicted':np.squeeze(y_predicted)
  })
  df_predicted.sort_values(by=['x'],inplace=True)
  df_predicted = df_predicted.round(10)
  df_predicted
  df_predicted_plot = df_predicted.melt(id_vars=['x'])
  return px.line(df_predicted_plot,x='x',y='value',color='variable')
plot_logreg(m,b)
# ! Plotly no puede graficar correctamente la curva cuando el valor de m es demasiado pequeño, lo muestra como una recta

## Resultados 

In [ ]:
ms = []
bs = []
js = []
for i in range(50): 
  temp_m, temp_b, temp_j = SGD(100, 0.01, 20)
  ms.append(temp_m)
  bs.append(temp_b)
  js.append(temp_j)
df_results = pd.DataFrame({
    'm': ms,
    'b': bs,
    'J': js
})
df_results.sort_values(by=['J'], ascending=False, inplace=True)
best_m = df_results.iloc[0]['m']
best_b = df_results.iloc[0]['b']
best_j = df_results.iloc[0]['J']
df_results.head()

,m,b,J
35,3.999241,3.000558,1.322333
18,3.993881,2.999580,1.320540
12,3.918191,1.984113,1.315703
9,3.912379,1.922912,1.315148
6,3.913814,1.960281,1.314744


Mejores valores encontrados

In [ ]:
# ! Plotly no puede graficar correctamente la curva cuando el valor de m es demasiado pequeño, lo muestra como una recta
plot_logreg(best_m,best_b)